# Vector boson fusion in IDM

The final notebook is [Significance_countours.ipynb](./Significance_countours.ipynb) which is based on [Sensitivity_fit.ipynb](Sensitivity_fit.ipynb) which is based on [Sensitivity_plots.ipynb](Sensitivity_plots.ipynb) by JD

For MadGraph tutorial see: https://goo.gl/exubD6

### Initialization

In [1]:
import sys
import subprocess
import numpy as np
import pandas as pd
##pip3 install pyslha
#import pyslha # not longer required
import tempfile
import os
import re
import nose
#General config: TODO, make config file and use standard module
MADGRAPH='madgraph' # configure MadGraph dir here!
NO_TEST=False

def grep(pattern,multilinestring):
    '''Grep replacement in python
    as in: $ echo $multilinestring | grep pattern
    dev: re.M is for multiline strings
    '''
    import re 
    grp=re.finditer('(.*)%s(.*)' %pattern, multilinestring,re.M)
    return '\n'.join([g.group(0) for g in grp])

def subprocess_line_by_line(*args,TRUST_ERRORS=True,**kwargs):
    '''
    Subprocess output line by line. Stop of error found when TRUST_ERRORS=True, and simply
    report wait method otherwise.
    
    The arguments are the same as for the Popen constructor.
    
    WARNING: Works only in Python 3
    
    See: https://stackoverflow.com/a/28319191/2268280 
    and: https://stackoverflow.com/a/17698359/2268280
    
    Example:
    
    subprocess_line_by_line('for i in $(seq 1 3);do echo $i; sleep 1;done',shell=True)
    '''
    
    if not TRUST_ERRORS:
        kwargs['stderr']=subprocess.PIPE
        
    kwargs['stdout']=subprocess.PIPE
    kwargs['bufsize']=1
    kwargs['universal_newlines']=True
    s=subprocess.Popen(*args,**kwargs)
    with s as p:
        for line in p.stdout:
            print(line, end='') # process line here
    
    if TRUST_ERRORS:
        if p.returncode != 0:
            raise subprocess.CalledProcessError(p.returncode, p.args)
    else:
        return s.wait()
    
##Main madGRAPH script:
def preamble(cfg):
    return '''import model '''+cfg.UFO_model+'''
define p = g u c d s b u~ c~ d~ s~ b~
define j = p  
define l+ = e+ mu+ 
define l- = e- mu- 
define vl = ve vm vt 
define vl~ = ve~ vm~ vt~

'''+cfg.processes+'''

output ../'''+cfg.work_dir+'''/'''+cfg.output_dir+'''

'''

def lamL_loop(MH0,MHc,MA0,lamL,cfg):
    mg5_script='launch ../{:s}/{:s}\n'.format(cfg.work_dir,cfg.output_dir)
    mg5_script=mg5_script+'0\n'
    mg5_script=mg5_script+'{:s}\n'.format(cfg.LHA_input_file)
    mg5_script=mg5_script+'{:s}\n'.format(cfg.Card_file)
    mg5_script=mg5_script+'set nevents {:d}\n'.format(cfg.number_of_events)
    if MH0>0:
        mg5_script=mg5_script+'set wa0 auto\n'
        mg5_script=mg5_script+'set whch auto \n'
        mg5_script=mg5_script+'set lamL {:s}\n'.format(str(lamL))
        mg5_script=mg5_script+'set mmh0 {:s}\n'.format(str(MH0))
        mg5_script=mg5_script+'set mma0 {:s}\n'.format(str(MA0))
        mg5_script=mg5_script+'set mmhch {:s}\n'.format(str(MHc))
    mg5_script=mg5_script+'\n'
    mg5_script=mg5_script+'0\n'
    
    return mg5_script

def closing(cfg):
    return '''launch ../'''+cfg.work_dir+'''/'''+cfg.output_dir+''' -i
print_results --path=./result_'''+cfg.output_dir+'''.txt --format=short


done
'''

def check_root_install(cfg):
    cfg=pd.Series(cfg)
    f=open('kk.sh','w')
    f.write('source '+cfg.thisroot+'\n')
    f.write('which root\n')
    f.close()
    
    if not subprocess.Popen('bash kk.sh'.split(),
                stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()[0]:
        sys.exit('INSTALL ROOT: see instrucctions in notebook')
        
def clone_git_repo(cfg,REPO='VBF_IDM',REPO_url='git@github.com:restrepo',git_options='--recursive'):
    cfg=pd.Series(cfg)
    if cfg.CLONE_GIT_REPO:  
        REPO
        REPO_url
         #WARNING: Try to overwirte contents
        if os.path.exists(cfg.main_dir+'index.ipynb'):
            sys.exit('ERROR: Repo files already exists. Check cfg.main_dir')
        if not os.path.isdir(cfg.main_dir):
            s=subprocess.Popen(['mkdir','-p',cfg.main_dir],stdout=subprocess.PIPE,stderr=subprocess.PIPE).wait()

        td=tempfile.mkdtemp()
        s=subprocess_line_by_line(('git clone  '+git_options+' '+REPO_url+'/'+REPO+'.git').split(),cwd=td,
                     stdout=subprocess.PIPE,stderr=subprocess.PIPE,TRUST_ERRORS=False)

        s=subprocess.Popen('mv '+td+'/'+REPO+'/*  '+cfg.main_dir,shell=True,
                       stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

        s=subprocess.Popen('mv '+td+'/'+REPO+'/.* '+cfg.main_dir,shell=True,
                       stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

        os.rmdir(td+'/'+REPO)
        os.rmdir(td)
    else:
        if cfg.VERBOSE:
            print('Skiping git clone')        
            
def install_pythia_delphes(cfg,release='v2.3.3'):
    if cfg.INSTALL:
        s=subprocess.Popen('git branch'.split(),cwd=cfg.MADGRAPH,
                              stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

        if not grep('\* '+release,s[0].decode('utf-8')):
            s=subprocess.Popen( ('git checkout -b '+release).split(),cwd=cfg.MADGRAPH,
                              stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
            if 'Switched' not in s[1].decode('utf-8'):
                sys.exit('Submodule problems')

        #subprocess does not use .bashrc        
        f=open(cfg.MADGRAPH+'/kk.sh','w')
        f.write('source '+cfg.thisroot+'\n')
        f.write('./bin/mg5_aMC install.dat\n')
        f.close()        
        if cfg.VERBOSE:
            subprocess_line_by_line('bash kk.sh'.split(),cwd=cfg.MADGRAPH, TRUST_ERRORS=False )
        else:
            s=subprocess.call('bash kk.sh'.split(),cwd=cfg.MADGRAPH, stdout=open('kk','w'),stderr=open('kkk','w') )
    else:
        if cfg.VERBOSE:
            print('Pythia: OK')
            print('Delphes: OK')  
                        
def not_html_opening(cfg):
    f=open(cfg.MADGRAPH+'/input/mg5_configuration.txt','r')
    mgc=f.read()
    f.close()

    f=open(cfg.MADGRAPH+'/input/mg5_configuration.txt','w')
    f.write(mgc.replace('# automatic_html_opening = True','automatic_html_opening = False'))
    f.close()           
            

def run_madgraph(MH0,MHc,MA0,LambdasL,cfg):
    # Prepare MadGraph-tools scripts directory
    s=subprocess.Popen(['mkdir','-p',cfg.work_dir+'/'+cfg.scripts_dir],
                     stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
    
    # Creates MadGraph script
    f=open(cfg.work_dir+'/'+cfg.scripts_dir+'/'+cfg.MadGraph_script,'w')
    f.write( preamble(cfg) )
    for lamL in LambdasL:
        f.write( lamL_loop(MH0,MHc,MA0,lamL,cfg) )
    f.write( closing(cfg) )
    f.close()

    # Prepare MadGraph launch command (requeries source thisroot.sh confifurations)
    f=open(cfg.MADGRAPH+'/kk.sh','w')
    f.write('source '+cfg.thisroot+'\n')
    f.write('./bin/mg5_aMC ../'+cfg.work_dir+'/'+cfg.scripts_dir+'/'+cfg.MadGraph_script+'\n')
    f.close()

    # launch command from MadGraph directory
    if not cfg.VERBOSE:
        s=subprocess.Popen( 'bash kk.sh'.split(), cwd=cfg.MADGRAPH,
                                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        s.wait()

    if cfg.VERBOSE:
        subprocess_line_by_line( 'bash kk.sh'.split(), cwd=cfg.MADGRAPH,TRUST_ERRORS=False)
        
def run_madevent(MH0,MHc,MA0,LambdasL,cfg):
    if len(LambdasL)>99:
        sys.exit('ERROR: UPDATE FORMAT FOR > 99 runs')
    # Prepares Pythia-Delphes script        
    f=open(cfg.work_dir+'/'+cfg.scripts_dir+'/'+cfg.pythia_script,'w')
    for r in range(1,len(LambdasL)+1):
        f.write('pythia run_%02d\n' %r)
        f.write('3\n')
        f.write(cfg.Delphes_card_file+'\n')
        f.write('0\n')
    f.close()

    # Prepare Pythia-Delphes launch command (requeries source thisroot.sh confifurations)
    f=open(cfg.work_dir+'/'+cfg.output_dir+'/kk.sh','w')
    f.write('source '+cfg.thisroot+'\n')
    f.write('./bin/madevent ../'+cfg.scripts_dir+'/'+cfg.pythia_script+'\n')
    
    f.close()

    # launch command from MadGraph-output directory
    (PHOUT,PHERR)=subprocess.Popen('bash kk.sh'.split(), cwd=cfg.work_dir+'/'+cfg.output_dir,
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

    #print(PHOUT.decode('utf-8'))
    DEBUG=True
    if DEBUG:
        f=open('PHOUT.txt','w')
        f.write(PHOUT.decode('utf-8'))
        f.close()

    # Analyse output to get the cross-section of each run
    cs_pb=np.array( re.sub( '\s+\+\-\s+[0-9\+\-eE\.]+\s+pb','\n',  
              re.sub('\s+Cross-section\s+:\s+','' ,
              ''.join( grep('Cross-section',PHOUT.decode('utf-8')).split('\n') 
              ) )  ).strip().split('\n')  ).astype(float)

    # Store cross section in a pandas DataFrame
    if len(cs_pb)==len(LambdasL):
        df=pd.DataFrame({'xs_'+str(int(MH0)):cs_pb,'laL':LambdasL})
        return df
    else:
        sys.exit('Error: missing cross section')
        return pd.Series()
    
def store_output(MH0,MHc,MA0,LambdasL,cfg):
    s=subprocess.Popen(['mkdir', '-p',cfg.full_output_dir]).wait()

    for r in range(1,len(LambdasL)+1):
        nrun='%02d' %r
        nrun3='%03d' %r
        s=subprocess.Popen(['cp',cfg.work_dir+'/'+cfg.output_dir+'/Events/run_'+nrun+'/tag_1_delphes_events.root', 
                                cfg.full_output_dir+'/delphes_events_'+str(int(MH0))+'_'+nrun3+'_.root'],
                                 stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if s.wait()>0:
            sys.exit('Files not found')

def merge_cross_sections_results(df,df_full=pd.read_csv('Output_data.csv')):
    dff=df_full.merge(df,on='laL',how='left').fillna(0)
    tmp=dff['laL']
    dff=dff.drop('laL',axis='columns')
    dff['laL']=tmp
    return dff

def main(scan_par,*input_par,only_config=False,skip_MadGraph=False,**cfg):
    '''
    Simulation of processes for each component of the scan_par list of the model through full chain 
    madgraph - Pythia -Delphes.
    
    Implemented model: Inert Doublet Model
    scan_par -> LambdaL list.
    input_par: List of the model parameters
        input_par[0] -> MH0
        input_par[1] -> MHc
        input_par[2] -> MA0
        
    Check main?? for **cfg options. 
    
    Default options are returned
    '''
    global MADGRAPH
    LambdasL=scan_par
    MH0=input_par[0]
    MHc=input_par[1]
    MA0=input_par[2]
    #Default values
    cfg=pd.Series(cfg)
    if 'thisroot' not in cfg:
        cfg['thisroot']='/home/restrepo/prog/ROOT/root/bin/thisroot.sh' 
        #cfg['thisroot']='/opt/root5/bin/thisroot.sh'
    if 'main_dir' not in cfg:
        cfg['main_dir']='.' # PATH of the .git of the cloned repository        
    if 'MADGRAPH' not in cfg:           
        cfg['MADGRAPH']=MADGRAPH # Name of the MadGraph installation. Configured at beggining        
    if 'Card_file' not in cfg:
        cfg['Card_file']='../Cards/run_card.dat'
    if 'number_of_events' not in cfg:
        cfg['number_of_events']=1000
    if 'UFO_model' not in cfg:
        cfg['UFO_model']='InertDoublet_UFO'
    if 'LHA_input_file' not in cfg:       
        cfg['LHA_input_file']='../MadGraph_cards/benchmarks/param_card_template.dat'
    #Loaded from cwd=cfg.work_dir+'/'+cfg.output_dir:
    if 'Delphes_card_file' not in cfg:       
        cfg['Delphes_card_file']='../../../Delphes_cards/delphes_card.dat'
    if 'processes' not in cfg:        
        cfg['processes']='generate p p > h2 h2 j j @0'    
        # To one specific (not gauge invariant contribution, see display diagrams) 
        #https://goo.gl/vG2W1h
        # GAUGE NOT INVARIANT:
        # generate p p > u u mu+ mu- QCD=0 QED=4 ;
        # p p > z > e+ e- #(ask one S-channel z)
        # p p > e+ e- / z #(forbids any z)
        # p p > e+ e- $$ z (forbids any z in s-channel)
        # generate p p > u u mu+ mu- QCD=0 QED=4  /a  #(combined)
        #SAFE:
        # p p > z, z > e+ e- (on-shell z decaying)
        # p p > e+ e- $ z (forbids s-channel z to be on-shell)
        #cfg['processes']='generate p p > h2 h2' 
        # p p > z, z > e+ e- (on-shell z decaying)
    if 'work_dir' not in cfg:        
        cfg['work_dir']='studies/IDM/' # Directory with the MadGraph-tools scripts        
    if 'scripts_dir' not in cfg:
        cfg['scripts_dir']='Task_Asana' # subdirectory of 'work_dir' with the MadGraph-tools scripts        
    if 'output_dir' not in cfg:        
        cfg['output_dir']='BP_'+str(int(MHc))+'_'+str(int(MH0))+'_vs_lambdaL' # MadGraph output subdirectory of work_dir
    if 'MadGraph_script' not in cfg:       
        cfg['MadGraph_script']='BP_'+str(int(MHc))+'_A_'+str(int(MH0))+'.txt' # MadGraph script
    if 'pythia_script' not in cfg:        
        cfg['pythia_script']='TemplateRunPythiaDelphes_all.dat' # Pythia-Delphes script
    if 'full_output_dir' not in cfg:
        cfg['full_output_dir'] ='output' # Final results dir for root and csv files
    if 'cross_sections_csv' not in cfg:
        cfg['cross_sections_csv']='cs_'+str(int(MHc))+'_'+str(int(MH0))+'.csv' # Final csv output
    if 'CLONE_GIT_REPO' not in cfg:        
        cfg['CLONE_GIT_REPO']=True #WARNING: Try to overwrite current contents!
    if 'INSTALL' not in cfg:          
        cfg['INSTALL']=False # If True check full installation
        if cfg.CLONE_GIT_REPO:
            cfg.INSTALL=True
    if 'VERBOSE' not in cfg:      
        cfg['VERBOSE']=True #Print shell commands output line by line 
        
    if only_config:
        return cfg
    print('========= Preparing run...====')
    if not skip_MadGraph:
        check_root_install(cfg)
        if not os.path.isdir('.git'): 
            clone_git_repo(cfg)
        if not os.path.isdir(MADGRAPH+'/pythia-pgs'): 
            install_pythia_delphes(cfg)
            
        not_html_opening(cfg)
    
        print('========= Runnig MadGraph (shown here if VERBOSE=True ====')
        run_madgraph(MH0,MHc,MA0,LambdasL,cfg)
        
    print('========= Runnig MadEvent. This can take a long...====')
    df=run_madevent(MH0,MHc,MA0,LambdasL,cfg)
    
    print('========= Saving root and csv files in :===='+cfg.full_output_dir+'...')
                    
    store_output(MH0,MHc,MA0,LambdasL,cfg)
    
    df.to_csv(cfg.full_output_dir+'/'+cfg.cross_sections_csv,index=False)
    
    print('======== CONGRATULATIONS for the successful runs =====')

    return cfg

def test_repo():
    if os.path.isdir('.git'): 
        return True
    global NO_TEST
    NO_TEST=True
    MHc=750
    MH0=240
    MA0=MHc
    LambdasL=[0.01]

    cfg=main(LambdasL,MH0,MHc,MA0,MadGraph_script='test.txt',processes='generate p p > h2 h2',work_dir='test',
                 output_dir='tmp',full_output_dir='test',cross_sections_csv='test.csv',VERBOSE=False,
                 CLONE_GIT_REPO=True,INSTALL=True)
    df=pd.read_csv(cfg.full_output_dir+'/'+cfg.cross_sections_csv)
    nose.tools.assert_almost_equal(df.xs_240.values[0],5.288E-8)


def test_install():
    global MADGRAPH
    if os.path.isdir(MADGRAPH+'/pythia-pgs'): 
        return True
    global NO_TEST
    NO_TEST=True
    MHc=750
    MH0=240
    MA0=MHc
    LambdasL=[0.01]

    cfg=main(LambdasL,MH0,MHc,MA0,MadGraph_script='test.txt',processes='generate p p > h2 h2',work_dir='test',
                 output_dir='tmp',full_output_dir='test',cross_sections_csv='test.csv',VERBOSE=False,
                 CLONE_GIT_REPO=True,INSTALL=True)
    df=pd.read_csv(cfg.full_output_dir+'/'+cfg.cross_sections_csv)
    nose.tools.assert_almost_equal(df.xs_240.values[0],5.288E-8)
    
def test_all():
    '''run with: 
       $ nosetest3 thisprogram.py 
       .
       ----------------------------------------------------------------------
       Ran 1 test in 73.094s

       OK
       
    It is assumed that repo is already cloned and MadGraph tools have been installed
    '''
    if NO_TEST:
        return True
    MHc=750
    MH0=240
    MA0=MHc
    LambdasL=[0.01]

    cfg=main(LambdasL,MH0,MHc,MA0,MadGraph_script='test.txt',processes='generate p p > h2 h2',work_dir='test',
                output_dir='tmp',full_output_dir='test',cross_sections_csv='test.csv',VERBOSE=False,
            CLONE_GIT_REPO=False,INSTALL=False)
    df=pd.read_csv(cfg.full_output_dir+'/'+cfg.cross_sections_csv)
    nose.tools.assert_almost_equal(df.xs_240.values[0],5.288E-8)
    


In [ ]:
if __name__=='__main__':
    MH0=400
    MHc=750
    MA0=MHc
    if MH0%1!=0:
        sys.exit('ERROR: MH0 must be integer')

    LambdasL=[0.01,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]
    
    cfg=main(LambdasL,MH0,MHc,MA0)

In [3]:
1+1

2

## Special runs

In [8]:
MH0=250
MHc=750
MA0=MHc
LambdasL=[0.01,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]
cfg=main(LambdasL,MH0,MHc,MA0,skip_MadGraph=True)

========= Runnig MadEvent. This can take a long...====
========= Saving root and csv files in :====output...
======== CONGRATULATIONS for the successful runs =====


### Signal and background

In [ ]:
if __name__=='__main__':
    VBF=True; MJ=False
    if not VBF:
        MJ=True
    BP={3:pd.Series({'MH0':65, 'MHc':200,'MA0':189.5,'LaL':0.009,'La2':0.1}),
        6:pd.Series({'MH0':65, 'MHc':500,'MA0':494,  'LaL':0.009,'La2':0.1}),
        7:pd.Series({'MH0':65, 'MHc':750,'MA0':750,  'LaL':0.009,'La2':0.1}),
        8:pd.Series({'MH0':65, 'MHc':750,'MA0':750,  'LaL':0.5,   'La2':0.1}),
        9:pd.Series({'MH0':110,'MHc':750,'MA0':750,  'LaL':0.009,'La2':0.1}),
        10:pd.Series({'MH0':0,'MHc':0,'MA0':0,  'LaL':0,'La2':0})
        }    
    N=10
    if MJ:
        N=9
    MH0=int(BP[N].MH0)
    MHc=int(BP[N].MHc)
    MA0=BP[N].MA0

    if MH0%1!=0 or MHc%1!=0:
        sys.exit('ERROR: MH0 and MHc must be integer')

    LambdasL=[BP[N].LaL]#,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]
    if MJ:
        LambdasL=[0.01,0.05,0.1,0.5,1.,2.]
    
    cfg=main(LambdasL,MH0,MHc,MA0,UFO_model='sm',processes='generate p p > vl vl~ j j @0',
             LHA_input_file='',Card_file='',Delphes_card_file='',number_of_events=1000000) 

## Collect cross section results

In [39]:
#cp Output_data_bak.csv Output_data.csv 

In [41]:
MH0s=np.arange(250,390,10)
df_full=pd.read_csv('Output_data.csv')
for M in MH0s:
    df=pd.read_csv('output/cs_{:d}_{:d}.csv'.format(MHc,M))
    df_full=merge_cross_sections_results(df,df_full)

In [46]:
df_full

,Unnamed: 0,xs_70,xs_75,xs_80,xs_85,xs_90,xs_110,xs_130,xs_150,xs_170,...,xs_300,xs_310,xs_320,xs_330,xs_340,xs_350,xs_360,xs_370,xs_380,laL
0,0,0.007836,0.007475,0.007075,0.006754,0.006451,0.005417,0.004505,0.003759,0.003205,...,0.001029,0.000943,0.000868,0.000799,0.000733,0.000670,0.000618,0.000567,0.000520,0.01
1,1,0.008207,0.007771,0.007350,0.006937,0.006623,0.005523,0.004599,0.003848,0.003219,...,0.001033,0.000951,0.000876,0.000803,0.000731,0.000671,0.000622,0.000572,0.000528,0.02
2,2,0.009770,0.008817,0.008136,0.007636,0.007198,0.005774,0.004747,0.003979,0.003283,...,0.001049,0.000961,0.000897,0.000824,0.000760,0.000695,0.000637,0.000584,0.000530,0.05
3,3,0.011421,0.009917,0.008990,0.008275,0.007629,0.006041,0.004918,0.004074,0.003398,...,0.001054,0.000973,0.000882,0.000816,0.000754,0.000695,0.000630,0.000575,0.000539,0.07
4,4,0.014126,0.011538,0.010077,0.009139,0.008302,0.006406,0.005106,0.004190,0.003460,...,0.001081,0.000995,0.000907,0.000827,0.000764,0.000695,0.000640,0.000585,0.000536,0.10
5,5,0.019924,0.015071,0.012550,0.010998,0.009767,0.007094,0.005427,0.004425,0.003637,...,0.001102,0.000994,0.000912,0.000834,0.000776,0.000706,0.000650,0.000601,0.000554,0.15
6,6,0.027188,0.019742,0.015811,0.013316,0.011713,0.007997,0.006022,0.004730,0.003859,...,0.001134,0.001036,0.000954,0.000883,0.000803,0.000733,0.000667,0.000615,0.000561,0.20
7,7,0.036839,0.025211,0.019486,0.016115,0.013757,0.008816,0.006455,0.005012,0.004055,...,0.001141,0.001048,0.000962,0.000876,0.000806,0.000753,0.000687,0.000624,0.000577,0.25
8,8,0.047742,0.031698,0.023874,0.019238,0.016198,0.009981,0.007122,0.005388,0.004225,...,0.001183,0.001081,0.000978,0.000891,0.000830,0.000748,0.000692,0.000631,0.000575,0.30
9,9,0.059759,0.039166,0.028822,0.022678,0.018853,0.011133,0.007755,0.005748,0.004519,...,0.001205,0.001105,0.001013,0.000930,0.000852,0.000777,0.000706,0.000653,0.000593,0.35


In [47]:
cfg=main(LambdasL,MH0,MHc,MA0,only_config=True)

In [51]:
df_full.to_csv('{:s}/Output_data.csv'.format(cfg.full_output_dir),index=False)

In [55]:
pd.read_csv('output/Output_data.csv').columns.values

array(['Unnamed: 0', 'xs_70', 'xs_75', 'xs_80', 'xs_85', 'xs_90', 'xs_110',
       'xs_130', 'xs_150', 'xs_170', 'xs_190', 'xs_210', 'xs_220',
       'xs_240', 'xs_250', 'xs_260', 'xs_270', 'xs_280', 'xs_290',
       'xs_300', 'xs_310', 'xs_320', 'xs_330', 'xs_340', 'xs_350',
       'xs_360', 'xs_370', 'xs_380', 'laL'], dtype=object)

## Appendix: Install root 5

### Prerequisites 
```bash
apt-get install cmake git dpkg-dev make g++ gcc binutils libx11-dev libxpm-dev \
libxft-dev libxext-dev gfortran libssl-dev libpcre3-dev \
xlibmesa-glu-dev libglew1.5-dev libftgl-dev \
libmysqlclient-dev libfftw3-dev libcfitsio-dev \
graphviz-dev libavahi-compat-libdnssd-dev \
libldap2-dev python-dev python3-dev libxml2-dev libkrb5-dev \
libgsl0-dev libqt4-dev r-base r-base-dev
```

Install ROOT 5 in some `PATH`
```bash
git clone http://root.cern.ch/git/root.git
cd root
git checkout v5-34-00-patches
./configure
make
```
Install pyroot:
```bash
cd PATH/root/compile/etc/notebook/kernels/
ipython kernelspec install root
```

Add to your `.bashrc`
```bash
source PATH/root/bin/thisroot.sh
```

##  Appendix: File results

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9]_*.root | more
delphes_events_70_001_.root
delphes_events_70_002_.root
delphes_events_70_003_.root
...
...
delphes_events_90_018_.root
delphes_events_90_019_.root
delphes_events_90_020_.root
```

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9][0-9]_*.root | more
delphes_events_110_001_.root
delphes_events_110_002_.root
delphes_events_110_003_.root
...
...
delphes_events_240_018_.root
delphes_events_240_019_.root
delphes_events_240_020_.root
```

cat readme.txt 
date 23/01/2017
Delphes output (.root) for the set the points contained in data_mh0_vs_x.dat

The data is organized as follows (in ascending order in lambda_L[lal] ):

#lal #mh0[GeV] #xs[fb]   #Delphes name
0.3 63.75 200        --> delphes_events_2.root
0.4 64.80 200        --> delphes_events_3.root 
0.5 66.32 200        --> delphes_events_4.root
0.6 68.05 200        --> delphes_events_5.root 
0.7 70.04 200        --> delphes_events_6.root
.   .     .          --> 
.   .     .          --> 
.   .     .          -->

The cross-section (xs) for the processes p p > h0 h0 j j
for all the set of points is around 200 fb. [Except for
the first point, with lal=0.3,  where the cross-section turns out to be 179 fb.]